<a href="https://colab.research.google.com/github/avikagupta03/DepressionTweetAnalyzer/blob/main/notebookf743c4044d.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:

# IMPORTANT: RUN THIS CELL IN ORDER TO IMPORT YOUR KAGGLE DATA SOURCES
# TO THE CORRECT LOCATION (/kaggle/input) IN YOUR NOTEBOOK,
# THEN FEEL FREE TO DELETE THIS CELL.
# NOTE: THIS NOTEBOOK ENVIRONMENT DIFFERS FROM KAGGLE'S PYTHON
# ENVIRONMENT SO THERE MAY BE MISSING LIBRARIES USED BY YOUR
# NOTEBOOK.

import os
import sys
from tempfile import NamedTemporaryFile
from urllib.request import urlopen
from urllib.parse import unquote, urlparse
from urllib.error import HTTPError
from zipfile import ZipFile
import tarfile
import shutil

CHUNK_SIZE = 40960
DATA_SOURCE_MAPPING = 'cdme-gan:https%3A%2F%2Fstorage.googleapis.com%2Fkaggle-data-sets%2F5786757%2F9507407%2Fbundle%2Farchive.zip%3FX-Goog-Algorithm%3DGOOG4-RSA-SHA256%26X-Goog-Credential%3Dgcp-kaggle-com%2540kaggle-161607.iam.gserviceaccount.com%252F20241007%252Fauto%252Fstorage%252Fgoog4_request%26X-Goog-Date%3D20241007T161701Z%26X-Goog-Expires%3D259200%26X-Goog-SignedHeaders%3Dhost%26X-Goog-Signature%3Db14c5b2cc71347690bc630966656ade670d884ffd3a861b7e16316a395e9b9ed2ec7e9ad6aaa372c011f84ee0e8e885631a1126ff2d28a97121df926126d41fc3602f6be3104c5b0db6c6176e75bbf7d7db24e22de7ff20bc117e1d2cb1d24dcb719b796a1d4b27f1db8ac0aecb7eeac7c58f5e2020f051d517bef564400b1425f391acd13bf62ab324859ecdf12b24199bb091ac07fb86617a9f8ab801a7f407883c3d449fb9a69961bb6f3fcb5980ac2fc16d863050f8f90d1a7c10aa688b875dd44ee4b9699d1813786ed2bf26ede321776b9cefc8dc0f7eb17ab7819ab8f493e7f27b546f3e853f2aab4c9e4e7225646a80677eb2b3a51f89cbba4b377ee'

KAGGLE_INPUT_PATH='/kaggle/input'
KAGGLE_WORKING_PATH='/kaggle/working'
KAGGLE_SYMLINK='kaggle'

!umount /kaggle/input/ 2> /dev/null
shutil.rmtree('/kaggle/input', ignore_errors=True)
os.makedirs(KAGGLE_INPUT_PATH, 0o777, exist_ok=True)
os.makedirs(KAGGLE_WORKING_PATH, 0o777, exist_ok=True)

try:
  os.symlink(KAGGLE_INPUT_PATH, os.path.join("..", 'input'), target_is_directory=True)
except FileExistsError:
  pass
try:
  os.symlink(KAGGLE_WORKING_PATH, os.path.join("..", 'working'), target_is_directory=True)
except FileExistsError:
  pass

for data_source_mapping in DATA_SOURCE_MAPPING.split(','):
    directory, download_url_encoded = data_source_mapping.split(':')
    download_url = unquote(download_url_encoded)
    filename = urlparse(download_url).path
    destination_path = os.path.join(KAGGLE_INPUT_PATH, directory)
    try:
        with urlopen(download_url) as fileres, NamedTemporaryFile() as tfile:
            total_length = fileres.headers['content-length']
            print(f'Downloading {directory}, {total_length} bytes compressed')
            dl = 0
            data = fileres.read(CHUNK_SIZE)
            while len(data) > 0:
                dl += len(data)
                tfile.write(data)
                done = int(50 * dl / int(total_length))
                sys.stdout.write(f"\r[{'=' * done}{' ' * (50-done)}] {dl} bytes downloaded")
                sys.stdout.flush()
                data = fileres.read(CHUNK_SIZE)
            if filename.endswith('.zip'):
              with ZipFile(tfile) as zfile:
                zfile.extractall(destination_path)
            else:
              with tarfile.open(tfile.name) as tarfile:
                tarfile.extractall(destination_path)
            print(f'\nDownloaded and uncompressed: {directory}')
    except HTTPError as e:
        print(f'Failed to load (likely expired) {download_url} to path {destination_path}')
        continue
    except OSError as e:
        print(f'Failed to load {download_url} to path {destination_path}')
        continue

print('Data source import complete.')


In [ ]:
import pandas as pd

In [ ]:
diagnosed_depression_path = '/kaggle/input/cdme-gan/I am diagnosed with depression.csv'  # 58 tweets
fighting_depression_path = '/kaggle/input/cdme-gan/I am fighting depression.csv'  # 80 tweets
suffering_depression_path = '/kaggle/input/cdme-gan/I suffer from depression.csv'  # 554 tweets
tweetdata_txt_path = '/kaggle/input/cdme-gan/tweetdata.txt'  # 160,454 tweets
depressive_processed_path = '/kaggle/input/cdme-gan/depressive_tweets_processed.csv'  # 2,323 tweets
random_tweets_path = '/kaggle/input/cdme-gan/sentiment140.csv'  # Sentiment140 dataset

In [ ]:
# Load depressive datasets
diagnosed_depression = pd.read_csv(diagnosed_depression_path, names=['tweet_text'])
fighting_depression = pd.read_csv(fighting_depression_path, names=['tweet_text'])
suffering_depression = pd.read_csv(suffering_depression_path, names=['tweet_text'])

In [ ]:
diagnosed_depression.head(10)

,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,tweet_text
id,conversation_id,created_at,date,time,timezone,user_id,username,name,place,tweet,language,mentions,urls,photos,replies_count,retweets_count,likes_count,hashtags,cashtags,link,retweet,quote_url,video,thumbnail,near,geo,source,user_rt_id,user_rt,retweet_id,reply_to,retweet_date,translate,trans_src,trans_dest
1434955769229754373,1434955769229754373,2021-09-06 19:04:50 UTC,2021-09-06,19:04:50,+0000,1381318737496838152,andreeazxm,andreea🤎,NaN,Tw: depression I was diagnosed with depression for a year and no one from my family knows that. I thought things were getting better till today. I was scared that I can hurt myself in some way. I want to tell my mom but I am scared to see her reactions and I dont know what to do.,en,[],[],[],3,0,1,[],[],https://twitter.com/andreeazxm/status/1434955769229754373,False,NaN,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,[],NaN,NaN,NaN,NaN
1434945519395106816,1434945519395106816,2021-09-06 18:24:06 UTC,2021-09-06,18:24:06,+0000,757411816079134720,generouspanda,☕Dàfāng (大方)☕,NaN,"To all the people checking on me yesterday, thank you. For those who don't know I am clinically diagnosed with bi-polar depression and while I have it under control for the most part, this past weekend it spiked hard. I'm doing better today. Thank you all, I don't deserve you.",en,[],[],[],2,0,7,[],[],https://twitter.com/GenerousPanda/status/1434945519395106816,False,NaN,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,[],NaN,NaN,NaN,NaN
1434869483366088708,1434869483366088708,2021-09-06 13:21:58 UTC,2021-09-06,13:21:58,+0000,408163969,annbag2,Ann Sproule🌏🔥💧😷,NaN,"What gives you strength and bearing to cope with a young adult with BPD, depression, self harm, most probably (but diagnosed) asd, suicidal, abusive, violent. Yes, we've had heaps of professional help. I am almost at my whits end. So sad myself. I love her and I hate her. FMD",en,[],[],[],4,0,7,[],[],https://twitter.com/annbag2/status/1434869483366088708,False,NaN,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,[],NaN,NaN,NaN,NaN
1434817177819963394,1434709252363390983,2021-09-06 09:54:07 UTC,2021-09-06,09:54:07,+0000,1317853886871371776,moogmarion,Marion,NaN,"@SteveWCarlson @12News He was diagnosed with transient schizophrenia, bipolar disorder, depression and anxiety, the same navy psychologists diagnosed in 2006. Jake is my cousins friend and I am following his story for a long time, Judge Lamberth is a Reagan appointee by the way.",en,[],[],[],3,0,0,[],[],https://twitter.com/MoogMarion/status/1434817177819963394,False,NaN,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,"[{'screen_name': 'SteveWCarlson', 'name': 'Senator Steve Carlson 2.022', 'id': '174369590'}, {'screen_name': '12News', 'name': '12 News', 'id': '17187236'}]",NaN,NaN,NaN,NaN
1434774860547330049,1434774860547330049,2021-09-06 07:05:58 UTC,2021-09-06,07:05:58,+0000,759164851319021569,discordkittem,meow for me,NaN,i have self diagnosed myself with depression. i am an edgy depressed emo 14 yr old girl. dhmu only real ones know😔🥀💔 https://t.co/ea4kVFwds1,en,[],[],['https://pbs.twimg.com/media/E-lXVVPVgAEg461.png'],5,0,25,[],[],https://twitter.com/discordkittem/status/1434774860547330049,False,NaN,1,https://pbs.twimg.com/media/E-lXVVPVgAEg461.png,NaN,NaN,NaN,NaN,NaN,NaN,[],NaN,NaN,NaN,NaN
1434707385159802882,1434678401986711554,2021-09-06 02:37:50 UTC,2021-09-06,02:37:50,+0000,1427264476047319055,metal_fuzz,MetalFuzz,NaN,"@DollBrat1985 Thank you for reaching out. You are so loved and cared about. That is a big step, I am proud of you. I fought mine for so long and I got diagnosed last year with depression and anxiety. I hated hearing those words, but I was able to put a name on my monster I was fighting. 😘🙏👏",en,[],[],[],0,0,1,[],[],https://twitter.com/metal_fuzz/status/1434707385159802882,False,NaN,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,"[{'screen_name': 'DollBrat1985', 'name': '🎀 Doll 🎀', 'id': '1256730189117173760'}]",NaN,NaN,NaN,NaN
1434664575979253760,1434664574381219841,2021-09-05 23:47:44 UTC,2021-09-05,23:47:44,+0000,1253742

In [ ]:
fighting_depression.head(10)

,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,tweet_text
id,conversation_id,created_at,date,time,timezone,user_id,username,name,place,tweet,language,mentions,urls,photos,replies_count,retweets_count,likes_count,hashtags,cashtags,link,retweet,quote_url,video,thumbnail,near,geo,source,user_rt_id,user_rt,retweet_id,reply_to,retweet_date,translate,trans_src,trans_dest
1434912344874766336,1434912342228176897,2021-09-06 16:12:17 UTC,2021-09-06,16:12:17,+0000,346387825,vizio32,Daddy Diddlez,NaN,I have been fighting my depression as it got really bad during my last job from being way over worked and from being told i was doing a good job one day and then the next day being told i am a crappy worker just cause i told them no about working over time everyday.,en,[],[],[],1,0,1,[],[],https://twitter.com/vizio32/status/1434912344874766336,False,NaN,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,[],NaN,NaN,NaN,NaN
1434845574801641473,1434845574801641473,2021-09-06 11:46:57 UTC,2021-09-06,11:46:57,+0000,1023981894,tumisangmo,Champagne Mami,NaN,"I don’t wanna lie ,for the past 3 days I have been myself for some hours in a day and it feels so good. I am greatful for small wins 😊🥲❤️🌞. Fighting depression is one thing il make sure I don’t ever come back to. I pray a lot !",en,[],[],[],0,0,1,[],[],https://twitter.com/TumisangMo/status/1434845574801641473,False,NaN,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,[],NaN,NaN,NaN,NaN
1434707385159802882,1434678401986711554,2021-09-06 02:37:50 UTC,2021-09-06,02:37:50,+0000,1427264476047319055,metal_fuzz,MetalFuzz,NaN,"@DollBrat1985 Thank you for reaching out. You are so loved and cared about. That is a big step, I am proud of you. I fought mine for so long and I got diagnosed last year with depression and anxiety. I hated hearing those words, but I was able to put a name on my monster I was fighting. 😘🙏👏",en,[],[],[],0,0,1,[],[],https://twitter.com/metal_fuzz/status/1434707385159802882,False,NaN,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,"[{'screen_name': 'DollBrat1985', 'name': '🎀 Doll 🎀', 'id': '1256730189117173760'}]",NaN,NaN,NaN,NaN
1434652964438921231,1434652964438921231,2021-09-05 23:01:35 UTC,2021-09-05,23:01:35,+0000,4836449714,boughscat,whiskers,NaN,I am fighting demons rn. Fighting off depression with beer. Probably will close in on 100 drinks this weekend. And that’s not me bragging it’s me begging for help. Where is my wife,en,[],[],[],1,0,1,[],[],https://twitter.com/boughscat/status/1434652964438921231,False,NaN,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,[],NaN,NaN,NaN,NaN
1434600535076769796,1434600535076769796,2021-09-05 19:33:15 UTC,2021-09-05,19:33:15,+0000,543177827,original_lexi,Lexi,NaN,It is SO nice outside and probably only have a few of these days left until shit season. And I am literally fighting with myself to get my ass up and take my kids outside so they can enjoy it. Feeling like a shit mom. Depression really really sucks.,en,[],[],[],0,0,0,[],[],https://twitter.com/original_lexi/status/1434600535076769796,False,NaN,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,[],NaN,NaN,NaN,NaN
1434596133284745230,1434578534274527239,2021-09-05 19:15:46 UTC,2021-09-05,19:15:46,+0000,2205444827,cardoor64,💔,NaN,@Hideki_Naganuma I rarely tweet and I am still fighting crippling depression Sensei naganuma,en,[],[],[],1,1,2,[],[],https://twitter.com/CarDoor64/status/1434596133284745230,False,NaN,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,"[{'screen_name': 'Hideki_Naganuma', 'name': 'HIDEKI NAGANUMA｜CEO OF FUNKY FRESH BEATS', 'id': '150522620'}]",NaN,NaN,NaN,NaN
1434590982159233024,1434590982159233024,2021-09-05 18:55:18 UTC,2021-09-05,18:55:18,+0000,1263372377842282496,crammmmmmmmm,Priston Galzamio Pajallares,NaN,"Those times I created my walls, created my place, crying every night, having the anxiety and depression, wanted to finish my life...Here I am right now, on the straight road, having good progress. I grew, in every aspect. Who knows what will happen after a tiring day? Fighting!",en,[],[],[],0,0,0,[],[],https://twitter.com/Crammmmmmmmm/status/1434590982159233024,False,NaN,0,NaN,NaN,

In [ ]:
suffering_depression.head(10)

,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,tweet_text
id,conversation_id,created_at,date,time,timezone,user_id,username,name,place,tweet,language,mentions,urls,photos,replies_count,retweets_count,likes_count,hashtags,cashtags,link,retweet,quote_url,video,thumbnail,near,geo,source,user_rt_id,user_rt,retweet_id,reply_to,retweet_date,translate,trans_src,trans_dest
1434999854221209603,1434999854221209603,2021-09-06 22:00:00 UTC,2021-09-06,22:00:00,+0000,366063848,trojanviper,Often Ryan ~ OccasionallyEmma ~ ConstantlyEvolving,NaN,"Do you ""go through""( I don't say, ""suffer from depression""), anxiety? https://t.co/ri2UizESdW",en,[],['https://fb.watch/7Si4gwG8Ha/'],[],0,0,0,[],[],https://twitter.com/TrojanViper/status/1434999854221209603,False,NaN,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,[],NaN,NaN,NaN,NaN
1434991368296927234,1434973578630422536,2021-09-06 21:26:17 UTC,2021-09-06,21:26:17,+0000,1410047691388825605,whiteyy18lover,whiteyy18lover💙👄,NaN,"@CarlaRenae31 Sister....I do that too. I suffer from severe depression, anxiety and PTSD and I have always used humor (usually sarcastic humor 😂) to deal with what I'm feeling. You sound just like me girl. Are we twins 😂lol",en,[],[],[],1,0,1,[],[],https://twitter.com/whiteyy18lover/status/1434991368296927234,False,NaN,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,"[{'screen_name': 'CarlaRenae31', 'name': 'CarlaRenae831', 'id': '1412559795983470595'}]",NaN,NaN,NaN,NaN
1434990851122294784,1434990851122294784,2021-09-06 21:24:14 UTC,2021-09-06,21:24:14,+0000,1353858144238145536,ihlersheri,Sheri Has Fucking Had It!,NaN,I suffer from Chronic Depression -- here's a song for my fellow sufferers... Jumper (1998 Edit) https://t.co/QuNZkWx6ut via @YouTubeMusic,en,"[{'screen_name': 'youtubemusic', 'name': 'youtube music', 'id': '880142286176231425'}]",['https://music.youtube.com/watch?v=tjK8zK3TMJc&feature=share'],[],0,0,1,[],[],https://twitter.com/IhlerSheri/status/1434990851122294784,False,NaN,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,[],NaN,NaN,NaN,NaN
1434988985533747205,1434959258127421442,2021-09-06 21:16:49 UTC,2021-09-06,21:16:49,+0000,1295300897350787072,ruslunam_art,Russ 🐺🌕,NaN,"@StarboyNear I suffer from depression and am really struggling everyday just to be here, however many people seem to take joy in making everything harder and hurting me. I want to move on but problems keep adding up and I'm exhausted.",en,[],[],[],0,0,0,[],[],https://twitter.com/RusLunam_art/status/1434988985533747205,False,NaN,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,"[{'screen_name': 'StarboyNear', 'name': '𝔑𝔢𝔞𝔯𝔪𝔬𝔬𝔫', 'id': '4860493286'}]",NaN,NaN,NaN,NaN
1434987168997408769,1434984360189435904,2021-09-06 21:09:36 UTC,2021-09-06,21:09:36,+0000,487634550,zycr9,Zycr9👈👦👉#BlueWave✊🇺🇸✊,NaN,@girlconundrum When U feel like 👆please watch something you like or listen to Ur favorite music or go for a walk or talk to Ur friends &amp; family. But please keep your mind busy 🙏😔 I suffer from depression &amp; I feel like 👆 all the time. So I keep trying different things to escape my thoughts,en,[],[],[],0,0,1,[],[],https://twitter.com/zycr9/status/1434987168997408769,False,NaN,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,"[{'screen_name': 'girlconundrum', 'name': 'Girl Conundrum 💙🌊#VaxMeAgain!💉', 'id': '3311799854'}]",NaN,NaN,NaN,NaN
1434979755967746053,1434911719646707716,2021-09-06 20:40:09 UTC,2021-09-06,20:40:09,+0000,1380641789862764544,helena_boxing,BoxingHelena,NaN,@geostylegeo New research will always change the way we think about depression! I think the more people who are educated about depression the better off they will be in understanding about depression! People who suffer from depression need extra care and unconditional love!,en,[],[],[],0,0,0,[],[],https://twitter.com/helena_boxing/status/1434979755967746053,False,NaN,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,"[{'screen_name': 'geostylegeo', 'name': 'George Styles', 'id': '18584590'}]",NaN,NaN,NaN,NaN
1434971204188876806,1434963615891722256,2021-09-06 20:06:10 UTC,2021-09-06,20:06:10,+0000,70669025,bakednotfried1,baked ⛽,NaN,@TrixieReaper 

In [ ]:
with open(tweetdata_txt_path, 'r', encoding='utf-8') as file:
    tweetdata_txt = pd.DataFrame(file.readlines(), columns=['tweet_text'])

In [ ]:
tweetdata_txt['tweet_text'] = tweetdata_txt['tweet_text'].str.strip()

In [ ]:
print(tweetdata_txt)

                                               tweet_text
0       {"created_at":"Sat May 16 18:12:31 +0000 2020"...
1                                                        
2       {"created_at":"Sat May 16 18:12:31 +0000 2020"...
3                                                        
4       {"created_at":"Sat May 16 18:12:31 +0000 2020"...
...                                                   ...
320903                                                   
320904  {"created_at":"Sat May 16 20:49:56 +0000 2020"...
320905                                                   
320906  {"created_at":"Sat May 16 20:49:56 +0000 2020"...
320907                                                   

[320908 rows x 1 columns]


In [ ]:
depressive_processed = pd.read_csv(depressive_processed_path, names=['tweet_text'])

In [ ]:
depressive_processed.head(10)

,,,,,,,,,,,,,,,,,,,,,,,,,tweet_text
989292962323615744|2018-04-25|23:59:57|Eastern Standard Time|whosalli|The lack of this understanding is a small but significant part of what causes anxiety & depression to both feel so incredibly lonely. It's soooo easy to compare. It's so easy to invalidate ourselves because of that.|1|0|3|,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
989292959844663296|2018-04-25|23:59:56|Eastern Standard Time|estermnunes|i just told my parents about my depression and it's so hard to get gen x people to understand that this is not something that i can control all the time or just cure with a walk or by keeping my mind busy|1|0|2|,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
989292951716155392|2018-04-25|23:59:54|Eastern Standard Time|TheAlphaAries|depression is something i don't speak about even going through it because it's also such a double edged sword. i love every race. even if white people had done so much i can't hate them all. my grandma is legit white! how tf can i hate,i do dislike people though.|0|0|0|,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
989292873664393218|2018-04-25|23:59:35|Eastern Standard Time|_ojhodgson|Made myself a tortilla filled with pb&j. My depression is cured. Olivia:1 depression:0|1|0|0|,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
989292856119472128|2018-04-25|23:59:31|Eastern Standard Time|DMiller96371630|@WorldofOutlaws I am gonna need depression meds soon,these rainouts are spinning my equilibrium out <Emoji: Pouting face> Mother Nature is being a mean B-T-H <Emoji: Face with symbols over mouth>|0|0|0|,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
989292843125616641|2018-04-25|23:59:28|Eastern Standard Time|PHOTOGPRODIG|my anxiety and my depression fighting over whos day it is https://twitter.com/crissles/status/988218861697806336 …|0|0|2|,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
989291000911642625|2018-04-25|23:52:09|Eastern Standard Time|sp4cecounty|wow she's suddenly cured my depression and gave us world peace https://twitter.com/bearfaceingtons/status/989289124883566593 …|0|2|9|,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
989290997346643968|2018-04-25|23:52:08|Eastern Standard Time|sassi_klassi|I am officially done with @kanyewest. him,the neptunes,justin timberlake,timbaland. got me out of depression when I was in younger. a lot of these artist have changed their sounds and I have still supported them,but this is too far|0|0|0|,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
989290972973600770|2018-04-25|23:52:02|Eastern Standard Time|therealazukaobi|Me: what's wrong?My girl: *looks up at me with look of depression and stress on her face**Me: same babe same. #examseason|0|0|2|#examseason,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
989290951012044800|2018-04-25|23:51:57|Eastern Standard Time|KateSusabu|@AusBorderForce @PeterDutton_MP @shanebazzi Agreed <Emoji: Clapping hands sign><Emoji: Clapping hands sign><Emoji: Clapping hands sign>As someone with long experience of clinical depression I'd like to add this:Of course she fuckingwell yells and hits her head on the wall and says she wants to dieNobody can live forever without hopeShe won't get better while kept indefinitely on Nauru|0|0|0|,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [ ]:
# Combine the depressive datasets
depressed_df = pd.concat([
    diagnosed_depression,
    fighting_depression,
    suffering_depression,
    tweetdata_txt,
    depressive_processed
])

In [ ]:
depressed_df.head()

,tweet_text
"(id, conversation_id, created_at, date, time, timezone, user_id, username, name, place, tweet, language, mentions, urls, photos, replies_count, retweets_count, likes_count, hashtags, cashtags, link, retweet, quote_url, video, thumbnail, near, geo, source, user_rt_id, user_rt, retweet_id, reply_to, retweet_date, translate, trans_src)",trans_dest
"(1434955769229754373, 1434955769229754373, 2021-09-06 19:04:50 UTC, 2021-09-06, 19:04:50, +0000, 1381318737496838152, andreeazxm, andreea🤎, nan, Tw: depression I was diagnosed with depression for a year and no one from my family knows that. I thought things were getting better till today. I was scared that I can hurt myself in some way. I want to tell my mom but I am scared to see her reactions and I dont know what to do., en, [], [], [], 3, 0, 1, [], [], https://twitter.com/andreeazxm/status/1434955769229754373, False, nan, 0, nan, nan, nan, nan, nan, nan, nan, [], nan, nan, nan)",NaN
"(1434945519395106816, 1434945519395106816, 2021-09-06 18:24:06 UTC, 2021-09-06, 18:24:06, +0000, 757411816079134720, generouspanda, ☕Dàfāng (大方)☕, nan, To all the people checking on me yesterday, thank you. For those who don't know I am clinically diagnosed with bi-polar depression and while I have it under control for the most part, this past weekend it spiked hard. I'm doing better today. Thank you all, I don't deserve you., en, [], [], [], 2, 0, 7, [], [], https://twitter.com/GenerousPanda/status/1434945519395106816, False, nan, 0, nan, nan, nan, nan, nan, nan, nan, [], nan, nan, nan)",NaN
"(1434869483366088708, 1434869483366088708, 2021-09-06 13:21:58 UTC, 2021-09-06, 13:21:58, +0000, 408163969, annbag2, Ann Sproule🌏🔥💧😷, nan, What gives you strength and bearing to cope with a young adult with BPD, depression, self harm, most probably (but diagnosed) asd, suicidal, abusive, violent. Yes, we've had heaps of professional help. I am almost at my whits end. So sad myself. I love her and I hate her. FMD, en, [], [], [], 4, 0, 7, [], [], https://twitter.com/annbag2/status/1434869483366088708, False, nan, 0, nan, nan, nan, nan, nan, nan, nan, [], nan, nan, nan)",NaN
"(1434817177819963394, 1434709252363390983, 2021-09-06 09:54:07 UTC, 2021-09-06, 09:54:07, +0000, 1317853886871371776, moogmarion, Marion, nan, @SteveWCarlson @12News He was diagnosed with transient schizophrenia, bipolar disorder, depression and anxiety, the same navy psychologists diagnosed in 2006. Jake is my cousins friend and I am following his story for a long time, Judge Lamberth is a Reagan appointee by the way., en, [], [], [], 3, 0, 0, [], [], https://twitter.com/MoogMarion/status/1434817177819963394, False, nan, 0, nan, nan, nan, nan, nan, nan, nan, [{'screen_name': 'SteveWCarlson', 'name': 'Senator Steve Carlson 2.022', 'id': '174369590'}, {'screen_name': '12News', 'name': '12 News', 'id': '17187236'}], nan, nan, nan)",NaN


In [ ]:
# Load the Sentiment140 dataset with proper column names
random_tweets = pd.read_csv(random_tweets_path, names=['sentiment', 'id', 'date', 'query', 'user', 'tweet_text'], encoding='latin1', skipinitialspace=True)


In [ ]:
print("Columns in diagnosed_depression dataset:", diagnosed_depression.columns)
print("Columns in fighting_depression dataset:", fighting_depression.columns)
print("Columns in suffering_depression dataset:", suffering_depression.columns)
print("Columns in depressive_processed dataset:", depressive_processed.columns)
print("Columns in random_tweets dataset:", random_tweets.columns)

Columns in diagnosed_depression dataset: Index(['tweet_text'], dtype='object')
Columns in fighting_depression dataset: Index(['tweet_text'], dtype='object')
Columns in suffering_depression dataset: Index(['tweet_text'], dtype='object')
Columns in depressive_processed dataset: Index(['tweet_text'], dtype='object')
Columns in random_tweets dataset: Index(['sentiment', 'id', 'date', 'query', 'user', 'tweet_text'], dtype='object')


In [ ]:
depressed_df['label'] = 1

In [ ]:
# Create Depressive Tweets Mega Dataset with a maximum of 50,000 tweets
if len(depressed_df) > 50000:
    depressed_df = depressed_df.sample(n=50000, random_state=42).reset_index(drop=True)

In [ ]:
non_depressed_df = random_tweets[random_tweets['sentiment'] == 4].reset_index(drop=True)

In [ ]:
# Check how many non-depressed tweets are available
total_positive_tweets = len(non_depressed_df)
print(f"Total positive tweets available: {total_positive_tweets}")

Total positive tweets available: 800000


In [ ]:
# Randomly select 50,000 non-depressed tweets if available
if total_positive_tweets >= 50000:
    non_depressed_df = non_depressed_df.sample(n=50000, random_state=42).reset_index(drop=True)
elif total_positive_tweets == 0:
    print("No positive tweets available in the dataset.")
    non_depressed_df = pd.DataFrame(columns=['tweet_text', 'label'])  # Create an empty DataFrame for non-depressed tweets


In [ ]:
# Add labels to both datasets
depressed_df['label'] = 1  # Label for depressed tweets
non_depressed_df['label'] = 0  # Label for non-depressed tweets

In [ ]:
# Combine the final dataset
final_dataset = pd.concat([depressed_df, non_depressed_df]).reset_index(drop=True)

# Shuffle the final dataset to ensure random distribution of classes
final_dataset = final_dataset.sample(frac=1, random_state=42).reset_index(drop=True)

In [ ]:
final_dataset.to_csv('balanced_final_dataset.csv', index=False)

In [ ]:
print(final_dataset)

                                              tweet_text  label  sentiment  \
0                           @stargazer60 that's awesome       0        4.0   
1                       @cunningstunts22  yes... i am!        0        4.0   
2      {"created_at":"Sat May 16 19:29:04 +0000 2020"...      1        NaN   
3      @Thorney88 re guinea fowl ive heard that lots ...      0        4.0   
4                        def going to the movies tonite       0        4.0   
...                                                  ...    ...        ...   
99995                                                         1        NaN   
99996  @DavidArchie so last night I made a hand-made ...      0        4.0   
99997  @nnkiani ah fair enough... i would watch seaso...      0        4.0   
99998  {"created_at":"Sat May 16 20:22:20 +0000 2020"...      1        NaN   
99999  {"created_at":"Sat May 16 19:11:40 +0000 2020"...      1        NaN   

                 id                          date     query    

In [ ]:
print(f"Final Dataset created with {len(final_dataset)} rows.")

Final Dataset created with 100000 rows.


In [ ]:
print("\nClass distribution in final dataset:")
print(f"Depressive Tweets Mega Dataset: {len(depressed_df)} tweets labeled as depressed.")
print(f"Random Tweets Dataset: {len(non_depressed_df)} tweets labeled as non-depressed.")


Class distribution in final dataset:
Depressive Tweets Mega Dataset: 50000 tweets labeled as depressed.
Random Tweets Dataset: 50000 tweets labeled as non-depressed.


In [ ]:
# Check class distribution in the final dataset
class_distribution = final_dataset['label'].value_counts()
print("Class distribution in the final dataset:")
print(class_distribution)


Class distribution in the final dataset:
label
0    50000
1    50000
Name: count, dtype: int64


In [ ]:
!pip install torch_geometric

In [ ]:
!pip install torch-scatter torch-sparse torch-cluster torch-spline-conv torch-geometric


In [ ]:
import re
import nltk
import torch
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from transformers import BertTokenizer, BertModel, RobertaTokenizer, RobertaModel, DebertaTokenizer, DebertaModel
from nltk.tokenize import word_tokenize
from nltk.corpus import stopwords
import torch.optim as optim
import torch.nn.functional as F
from torch_geometric.data import Data
from torch_geometric.loader import DataLoader
from torch_geometric.nn import GATConv
import torch.nn as nn

# Download NLTK resources
nltk.download('punkt')
nltk.download('stopwords')


[nltk_data] Downloading package punkt to /usr/share/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package stopwords to /usr/share/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


True

In [ ]:
# Preprocessing function to clean the tweets
def preprocess_tweet(tweet):
    if not isinstance(tweet, str):
        return ''
    tweet = re.sub(r'http\S+|www\S+', '', tweet)  # Remove URLs
    tweet = re.sub(r'@\w+', '', tweet)            # Remove mentions
    tweet = re.sub(r'[^A-Za-z\s]', '', tweet)     # Remove special characters and numbers
    tweet = tweet.lower()                         # Lowercase
    tweet_tokens = word_tokenize(tweet)
    stop_words = set(stopwords.words('english'))
    return ' '.join([word for word in tweet_tokens if word not in stop_words])


In [ ]:
def extract_embeddings_in_batches(texts, tokenizer, model, device, batch_size=32):
    model.to(device)  # Ensure the model is on the correct GPU
    embeddings = []

    for i in range(0, len(texts), batch_size):
        batch_texts = texts[i:i + batch_size]
        inputs = tokenizer(batch_texts, return_tensors='pt', truncation=True, padding=True, max_length=512).to(device)

        with torch.no_grad():
            outputs = model(**inputs)
            batch_embeddings = outputs.last_hidden_state.mean(dim=1).cpu().numpy()
            embeddings.append(batch_embeddings)

    return np.vstack(embeddings)


In [ ]:
# Assuming final_dataset is a pandas DataFrame containing 'tweet_text' and 'label'
final_dataset['cleaned_tweet'] = final_dataset['tweet_text'].apply(preprocess_tweet)

# Define features and labels
X = final_dataset['cleaned_tweet']  # Features (cleaned tweets)
y = final_dataset['label']  # Labels (1 for depressed, 0 for non-depressed)

# Split the dataset into training (80%) and testing (20%)
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42, stratify=y)


In [ ]:
device1 = torch.device('cuda:0')  # First GPU
device2 = torch.device('cuda:1')  # Second GPU

# Initialize BERT tokenizer and model
bert_tokenizer = BertTokenizer.from_pretrained('bert-base-uncased')
bert_model = BertModel.from_pretrained('bert-base-uncased').to(device1)  # BERT on GPU 1

roberta_tokenizer = RobertaTokenizer.from_pretrained('roberta-base')
roberta_model = RobertaModel.from_pretrained('roberta-base').to(device2)  # RoBERTa on GPU 2

deberta_tokenizer = DebertaTokenizer.from_pretrained('microsoft/deberta-base')
deberta_model = DebertaModel.from_pretrained('microsoft/deberta-base').to(device1)  # DeBERTa on GPU 1


/opt/conda/lib/python3.10/site-packages/transformers/tokenization_utils_base.py:1601: FutureWarning: `clean_up_tokenization_spaces` was not set. It will be set to `True` by default. This behavior will be depracted in transformers v4.45, and will be then set to `False` by default. For more details check this issue: https://github.com/huggingface/transformers/issues/31884
  warnings.warn(
Some weights of RobertaModel were not initialized from the model checkpoint at roberta-base and are newly initialized: ['roberta.pooler.dense.bias', 'roberta.pooler.dense.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [ ]:
# Extract embeddings for each model
X_train_embeddings_bert = extract_embeddings_in_batches(X_train.tolist(), bert_tokenizer, bert_model, device1)
X_test_embeddings_bert = extract_embeddings_in_batches(X_test.tolist(), bert_tokenizer, bert_model, device1)

X_train_embeddings_roberta = extract_embeddings_in_batches(X_train.tolist(), roberta_tokenizer, roberta_model, device2)
X_test_embeddings_roberta = extract_embeddings_in_batches(X_test.tolist(), roberta_tokenizer, roberta_model, device2)

X_train_embeddings_deberta = extract_embeddings_in_batches(X_train.tolist(), deberta_tokenizer, deberta_model, device1)
X_test_embeddings_deberta = extract_embeddings_in_batches(X_test.tolist(), deberta_tokenizer, deberta_model, device1)

# Average the embeddings from the three models
X_train_ensemble = (X_train_embeddings_bert + X_train_embeddings_roberta + X_train_embeddings_deberta) / 3
X_test_ensemble = (X_test_embeddings_bert + X_test_embeddings_roberta + X_test_embeddings_deberta) / 3


In [ ]:
# Save the embeddings to disk
np.save('X_train_ensemble_embeddings.npy', X_train_ensemble)
np.save('X_test_ensemble_embeddings.npy', X_test_ensemble)


In [ ]:
# Load the embeddings from disk
X_train_ensemble = np.load('X_train_ensemble_embeddings.npy')
X_test_ensemble = np.load('X_test_ensemble_embeddings.npy')


In [ ]:
# Function to create a fully connected edge index for n nodes
def fully_connected_edge_index(num_nodes):
    row = []
    col = []
    for i in range(num_nodes):
        for j in range(num_nodes):
            if i != j:
                row.append(i)
                col.append(j)
    edge_index = torch.tensor([row, col], dtype=torch.long)
    return edge_index

# Create edge_index for training and test data (fully connected graph)
train_edge_index = fully_connected_edge_index(X_train_ensemble.shape[0])
test_edge_index = fully_connected_edge_index(X_test_ensemble.shape[0])


In [ ]:
!zip -r /kaggle/working/embeddings.zip /kaggle/working/X_train_ensemble_embeddings.npy /kaggle/working/X_test_ensemble_embeddings.npy

  adding: kaggle/working/X_train_ensemble_embeddings.npy (deflated 27%)
  adding: kaggle/working/X_test_ensemble_embeddings.npy (deflated 27%)


In [ ]:
import shutil

# Move files to the output directory for download
shutil.move('X_train_ensemble_embeddings.npy', '/kaggle/working/X_train_ensemble_embeddings.npy')
shutil.move('X_test_ensemble_embeddings.npy', '/kaggle/working/X_test_ensemble_embeddings.npy')


'/kaggle/working/X_test_ensemble_embeddings.npy'

In [ ]:
import os
print(os.listdir('/kaggle/working'))


In [ ]:
!ls

In [ ]:
# Create Data objects for training and testing
train_data = Data(x=torch.tensor(X_train_ensemble, dtype=torch.float),
                  y=torch.tensor(y_train.values, dtype=torch.long),
                  edge_index=train_edge_index)

test_data = Data(x=torch.tensor(X_test_ensemble, dtype=torch.float),
                 y=torch.tensor(y_test.values, dtype=torch.long),
                 edge_index=test_edge_index)

# Split training data into training and validation sets
X_train, X_val, y_train, y_val = train_test_split(X_train_ensemble, y_train, test_size=0.2, random_state=42)

# Create edge_index for validation set (fully connected graph)
val_edge_index = fully_connected_edge_index(X_val.shape[0])

# Create Data objects for validation data
val_data = Data(x=torch.tensor(X_val, dtype=torch.float),
                y=torch.tensor(y_val.values, dtype=torch.long),
                edge_index=val_edge_index)

# Create DataLoader for training and validation data
train_loader = DataLoader([train_data], batch_size=1)
val_loader = DataLoader([val_data], batch_size=1)


In [ ]:
class GAT(nn.Module):
    def __init__(self, in_channels, hidden_channels, out_channels):
        super(GAT, self).__init__()
        self.conv1 = GATConv(in_channels, hidden_channels, heads=8, dropout=0.6)
        self.conv2 = GATConv(hidden_channels * 8, out_channels, heads=1, dropout=0.6)

    def forward(self, x, edge_index):
        x = self.conv1(x, edge_index)
        x = F.elu(x)
        x = self.conv2(x, edge_index)
        return F.log_softmax(x, dim=1)


In [ ]:
# Initialize the GAT model
model = GAT(in_channels=X_train_ensemble.shape[1], hidden_channels=64, out_channels=2).to(device1)

In [ ]:
# Define optimizer and loss function
optimizer = optim.Adam(model.parameters(), lr=0.005)
loss_fn = nn.NLLLoss()

In [ ]:
# Train function
def train(model, loader):
    model.train()
    epoch_loss = 0
    for batch in loader:
        optimizer.zero_grad()
        out = model(batch.x, batch.edge_index)
        loss = loss_fn(out, batch.y)
        loss.backward()
        optimizer.step()
        epoch_loss += loss.item()
    return epoch_loss / len(loader)


In [ ]:
# Evaluation function
def evaluate(model, loader):
    model.eval()
    total_loss = 0
    with torch.no_grad():
        for batch in loader:
            out = model(batch.x, batch.edge_index)
            loss = loss_fn(out, batch.y)
            total_loss += loss.item()
    return total_loss / len(loader)


In [ ]:
best_loss = float('inf')
patience = 20
counter = 0

for epoch in range(300):
    avg_loss = train(model, train_loader)
    val_loss = evaluate(model, val_loader)

    if val_loss < best_loss:
        best_loss = val_loss
        counter = 0
    else:
        counter += 1

    if counter >= patience:
        print(f"Early stopping at epoch {epoch}")
        break

    print(f'Epoch {epoch}, Loss: {avg_loss:.4f}, Validation Loss: {val_loss:.4f}')
